# Muhammad Wajahat Mirza

## NetID: mwm356

## MLLU Homework 01

# Part 1.

The deadline for Part 1 is **2 pm Feb 11, 2021**.   
You should submit a `.ipynb` file with your solutions to NYU Classes.

---
Spam filtering is a well-studied NLP classification problem that is used in many commercial products nowadays, including email clients and mobile SMS apps.

In this assignment we will train a logistic regression model to classify each text in the SMS Spam Collection dataset as either spam or legitimate. This dataset consists of 5,574 English SMS messages, each tagged as either spam or ham (legitimate). The column 'v1' contains the label and the column 'v2' contains the SMS text. We will pre-process and convert the texts into bag-of-words features for our model.

## Data Loading

First, we download the SMS Spam Collection Dataset. The below command downloads the dataset from [Kaggle](https://www.kaggle.com/uciml/sms-spam-collection-dataset/data#) and loads it to [Google Drive](https://drive.google.com/open?id=1OVRo37agn02mc6yp5p6-wtJ8Hyb-YMXR).

In [1]:
!wget 'https://docs.google.com/uc?export=download&id=1OVRo37agn02mc6yp5p6-wtJ8Hyb-YMXR' -O spam.csv 

--2021-02-11 15:09:29--  https://docs.google.com/uc?export=download&id=1OVRo37agn02mc6yp5p6-wtJ8Hyb-YMXR
Resolving docs.google.com (docs.google.com)... 64.233.189.101, 64.233.189.100, 64.233.189.138, ...
Connecting to docs.google.com (docs.google.com)|64.233.189.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-04-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ceklobioragack96ad6spr8833aj0jr7/1613056125000/08752484438609855375/*/1OVRo37agn02mc6yp5p6-wtJ8Hyb-YMXR?e=download [following]
--2021-02-11 15:09:30--  https://doc-14-04-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ceklobioragack96ad6spr8833aj0jr7/1613056125000/08752484438609855375/*/1OVRo37agn02mc6yp5p6-wtJ8Hyb-YMXR?e=download
Resolving doc-14-04-docs.googleusercontent.com (doc-14-04-docs.googleusercontent.com)... 108.177.125.132, 2404:6800:4008:c01::84
Connecting to doc-14-04-docs.googleusercontent.com (doc-14-04-

In [2]:
!ls

sample_data  spam.csv


## Import libraries

### This is to download NLTK packages. 

In [3]:



import nltk
import ssl

'''If you are using jupyter notebook, pop up screen will ask to download corpuses. 
   Once download is complete, close the pop up window to proceed'''
'''If you are using google Colab, just download first iteration of packages.
    Upon second action input bar, type q'''
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [

True

In [5]:
!pip install cleanco
import pandas as pd
import numpy as np
import re 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from cleanco import prepare_terms, basename
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data Import 

Now we preview the data. There are two columns: `v1` -- the label, which indicates whether the text is spam or ham (legitimate), and `v2` -- the text of the message.

In [6]:

'''Read the csv'''
df = pd.read_csv("spam.csv", usecols=["v1", "v2"], encoding='latin-1')
# 1 - spam, 0 - ham
df.v1 = (df.v1 == "spam").astype("int")
print("Data Size: ", df.shape[0])
df.head()

Data Size:  5572


,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


## Split the data using pandas and numpy

Your task is to randomly split the data into training, validation, and test sets. Make sure that each row appears in only one of the splits and that training contains 70% of the data, validation contains 15%, and test contains 15%. You can compute the size of each split afterwards to sanity check your code. **You may use numpy and pandas functions, but please do not use sklearn.**

In [7]:
# 0.15 for val, 0.15 for test, 0.7 for train

val_size = int(df.shape[0] * 0.15)
test_size = int(df.shape[0] * 0.15)
train_size = int(df.shape[0] * 0.70)

data = df.sample(frac=1, random_state=1)
print("\nSize of data without random sampling: {}\nSize of data with  random  sampling : {}\n"\
      .format(data.shape[0], df.shape[0]))

count_ham = 0
count_spam = 0
for i in range(data.shape[0]):
    if (data["v1"][i] == 0):
        count_ham += 1
    else:
        count_spam += 1

print("Data has total: \n{} HAM entries\n{} SPAM entries.\n\n{:.3}% of data is SPAM and {:.3}% is HAM\n"\
      .format(count_ham, count_spam, (count_spam/data.shape[0])*100, (count_ham/data.shape[0])*100))


Size of data without random sampling: 5572
Size of data with  random  sampling : 5572

Data has total: 
4825 HAM entries
747 SPAM entries.

13.4% of data is SPAM and 86.6% is HAM



### Verify Random Split and Random State

In [8]:
'''For testing purposes to verify that data split is random'''
'''Use Random State seed == 1 to make sure random split is same at each re-run '''
'''Reminder: 1 - spam, 0 - ham'''
print("\n=================Random Split with Seeding========================")
for i in range(2):
    data_test = df.sample(frac=1, random_state=None)
    print("\n",data_test.head(),"\n")
print("=================Seeding Random Split========================")
for i in range(2):
    data_test = df.sample(frac=1, random_state=1)
    print("\n",data_test.head(),"\n")


=================Random Split with Seeding========================

       v1                                                 v2
2522   0  Dunno lei... I might b eatin wif my frens... I...
3727   0  The search 4 happiness is 1 of d main sources ...
2677   0  Where in abj are you serving. Are you staying ...
4402   0                             Just getting back home
2462   0  Rose needs water, season needs change, poet ne... 


       v1                                                 v2
1251   1  Please CALL 08712402779 immediately as there i...
403    0   None of that's happening til you get here though
585    0  Tell them u have a headache and just want to u...
3344   0      Reverse is cheating. That is not mathematics.
2718   1  18 days to Euro2004 kickoff! U will be kept in... 

=================Seeding Random Split========================

       v1                                                 v2
1078   0                           Convey my regards to him
4028   0           [

### Verifying Data Split

In [9]:

train_set = data[:train_size]
test_set = data[train_size:test_size + train_size+1]
val_set = data[test_size + train_size + 1:]

print("Size of train data: {}\nSize of test data: {}\nSize of validate data: {}"\
      .format(train_set.shape[0],\
              test_set.shape[0],\
              val_set.shape[0]))
print("\nVerification of Data Split:\nPercentage breakdown\tTrain    = {:.3}%\
    \n\t\t\tTest     = {:.3}%\
    \n\t\t\tValidate = {:.3}%"\
      .format((train_set.shape[0]/df.shape[0])*100,\
               (test_set.shape[0]/df.shape[0])*100,\
               (val_set.shape[0]/df.shape[0])*100))


Size of train data: 3900
Size of test data: 836
Size of validate data: 836

Verification of Data Split:
Percentage breakdown	Train    = 70.0%    
			Test     = 15.0%    
			Validate = 15.0%


### Assigning Text Data and Label as per the split

In [10]:
'''Now, assign the values of text and label to each of the three split datasets'''

train_texts, train_labels = train_set["v2"], train_set["v1"]
val_texts, val_labels     = test_set["v2"], test_set["v1"]
test_texts, test_labels   = val_set["v2"], val_set["v1"]

print("\nData assignment verification (text size, label size):\n")
print("Train: ",train_texts.shape[0], train_labels.shape[0])
print("Test : ",val_texts.shape[0], val_labels.shape[0])
print("Validate: ",test_texts.shape[0], test_labels.shape[0])



Data assignment verification (text size, label size):

Train:  3900 3900
Test :  836 836
Validate:  836 836


## Data Processing

The task is to create bag-of-words features: tokenize the text, index each token, represent the sentence as a dictionary of tokens and their counts, limit the vocabulary to $n$ most frequent tokens. In Lab 2 we will use the built-in `sklearn` function, `sklearn.feature_extraction.text.CountVectorizer`. 
**In this HW, you are required to implement the class `Vectorizer` on your own without using `sklearn` built-in functions.**

Function `preprocess_data` takes the list of texts and returns list of (lists of tokens). 
You may use [spacy](https://spacy.io/) or [nltk](https://www.nltk.org/) text processing libraries in `preprocess_data` function. 

Class `Vectorizer` is used to vectorize the text and to create a matrix of features.


$\textbf{Process:}$

                    Punctuation were kept as they are indicative of SPAM
                    Tokens are all lowerCase
                    Stopwords are removed to adjust the Data Matrix

In [11]:

def remove_stopwords(tokenized_data):
    
    newText = []
    for word in tokenized_data:
        if word not in stopwords.words('english'):
            newText.append(word)
            
    return newText


def preprocess_data(data):
    # This function should return a list of lists of pre-processed tokens, where
    # each nested list contains the tokens for a single message. To pre-process
    # each message, tokenize the message and lowercase all text.
 
    tokenized_data = []
    for mesg in data:
        mesg = mesg.lower()
        
        tokenized_data.append(nltk.word_tokenize(mesg))
    
    
    ''' Removing stopwords from list of tokenized words'''
    preprocessed_data = []
    for i in range(len(tokenized_data)):
        without_stopwords = remove_stopwords(tokenized_data[i])
        preprocessed_data.append(without_stopwords)
    print("Verfiy the original length {} of data with preprocessed data: {}".format(len(data), len(preprocessed_data)))
    
    return preprocessed_data

In [12]:
print("\nPreprocessing Training Data:")
train_data = preprocess_data(train_texts)
print("\nPreprocessing Validating Data:")
val_data = preprocess_data(val_texts)
print("\nPreprocessing Test Data:")
test_data = preprocess_data(test_texts)


Preprocessing Training Data:
Verfiy the original length 3900 of data with preprocessed data: 3900

Preprocessing Validating Data:
Verfiy the original length 836 of data with preprocessed data: 836

Preprocessing Test Data:
Verfiy the original length 836 of data with preprocessed data: 836


### You can add more features to the feature matrix. 

(Not required, but worth extra credit points.)

### Functions for Additional Features

                Additional Features include:
                        Detecting a link in a Token
                        Checking for a Phone num
                        Seeing Company Name, indicative of SPAM in most cases
                        

In [13]:
def linkDetector(string): 
   
    return len([x[0] for x in \
                re.findall(r"(?i)\b((?:https?://|www\d{0,3}[.]|"
                           "[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|"
                           "\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\""
                           ".,<>?«»“”‘’]))",
                     string)] )


def company(business_name):
    terms = prepare_terms()
    name = basename(business_name, terms, prefix=True, middle=True, suffix=False)
    x = name.split(",")
    if len(x) <= 1:
        return False
    else:
        for term in terms:
            if x[1] in term[1]:
                print(x[1])
                return True
            else:
                return False

In [14]:
class Vectorizer():
    def __init__(self, max_features):
        self.max_features = max_features
        self.vocab_list = {}
        self.token_to_index = {}

    def fit(self, dataset):
        # Given a list of lists of tokens, create a vocab list, self.vocab_list, 
        # using the most frequent "max_features" tokens. 
        # Also create a token indexer, self.token_to_index, that will return the index 
        # of the token in self.vocab_list.
        
        '''Adding tokens to vocab_list along with their frequency'''
        for sentence in dataset:
            for word in sentence:
                if word not in self.vocab_list:
                    self.vocab_list[word] = 1
                else:
                    self.vocab_list[word] += 1
                    
        '''converting the frequency work dictionary to a tuple'''
        '''Sorting the list of tuples on freqency count'''
        '''Take the top Max_features Tokens based on frequency'''
        dictTotuple = [(word, count) for word, count in self.vocab_list.items()]  
        sorted_by_freq = sorted(dictTotuple, key=lambda tup: tup[1]) 
        
        
        self.vocab_list = []

        newlst = sorted_by_freq[-self.max_features:]
        
        
        for element in newlst:
            self.vocab_list.append(element[0])
        
        '''Creating the token_to_index for the top selected vocab'''
        for i in range(len(self.vocab_list)):
            self.token_to_index[self.vocab_list[i]] = i
        
        
    def transform(self, dataset):
        # This function transforms the text dataset (a list of lists of tokens) 
        # into a matrix, "data_matrix," where the entry located at (i, j) is 1 
        # if sample i of the dataset contains token j in the vocab list, 
        # and 0 otherwise.
        
        '''data_matrix = N * (max_features + d)'''
        data_matrix = np.zeros((len(dataset), len(self.vocab_list)+3))
        print("Data_Matrix Shape (N * (max_features + d)): ", data_matrix.shape)
    

        for i in range(len(dataset)):
            for j in range(len(dataset[i])):
                
                d = (dataset[i][j], 'utf-8')
                
                if (dataset[i][j] in self.vocab_list):
                    data_matrix[i][j] += 1
                    
                else:
                    data_matrix[i][j] = 0 
                    
                if (linkDetector(dataset[i][j]) > 0):
                    data_matrix[i][-1] = 1
                
                if ((d[0].isnumeric() == True and len(d[0]) > 3)):
                    data_matrix[i][-2] = 1
                    
                if (company(dataset[i][j]) == True):
                    data_matrix[i][-3] = 1
                    
        return data_matrix
    

### Call the Vectorizer on each dataset

In [15]:

max_features = 1000 # TODO: Replace None with a number
vectorizer = Vectorizer(max_features=max_features)
vectorizer.fit(train_data)

print("\nTransforming Training Data")
X_train = vectorizer.transform(train_data)
print("\nTransforming Validation Data")
X_val = vectorizer.transform(val_data)
print("\nTransforming Test Data")
X_test = vectorizer.transform(test_data)

'''setting labels to arrays'''
y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)

vocab = vectorizer.vocab_list



Transforming Training Data
Data_Matrix Shape (N * (max_features + d)):  (3900, 1003)

Transforming Validation Data
Data_Matrix Shape (N * (max_features + d)):  (836, 1003)

Transforming Test Data
Data_Matrix Shape (N * (max_features + d)):  (836, 1003)


## Model

We train a logistic regression model on the bag-of-words features and save predictions for the training, validation, and test sets.

In [16]:


# Define Logistic Regression model
'''Using l2 regularization since our Solver is liblinear'''
model = LogisticRegression(penalty='l2',random_state=0, solver='liblinear', max_iter=100, verbose=1)

# Fit the model to training data
model.fit(X_train, y_train)

# Make prediction using the trained model
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

[LibLinear]

## Performance of the model

Your task is to report train, val, test accuracies and F1 scores.
**You are required to implement `accuracy_score` and `f1_score` methods without using built-in python functions.**

Your model should achieve at least **0.95** test accuracy and **0.90** test F1 score.

### Solution: Performance Model
#### Accuracy, Precision, Recall, and F-1 Score

A true positive is an outcome where the model correctly predicts the positive class. Similarly, a true negative is an outcome where the model correctly predicts the negative class.

A false positive is an outcome where the model incorrectly predicts the positive class. And a false negative is an outcome where the model incorrectly predicts the negative class.

In my performance model, 
            
               if a message is correctly predicted HAM, it is True Positive.
               if a message is correctly predicted SPAM, it is True Negative.
               if a message is HAM but predicted as SPAM, it is False Negative.
               if a message is SPAM but predicted as HAM, it is False Positive.


$\textbf{Key:}$

                TP = True Positive
                TN = True Negative
                FN = False Negative
                FP = False Positive

$\textbf{Formulas:}$

$$Accuracy = \cfrac{TP + TN}{TP + TN + FN + FP}$$

$$Precision = \cfrac{TP}{TP + FP}$$


$$Recall = \cfrac{TP}{TP + FN}$$

$$F-1 Score = 2 * \cfrac{precision * recall}{precision + recall}$$

In [17]:
def calculate_confusion_matrix(ground_truth, prediction, datatype):
    '''
    Each correct ham               == true pos
    Each correct spam              == true neg
    Ham incorrectly labeled spam   == false neg
    Spam incorrectly labeled ham   == false pos
    '''
    # 1 - spam, 0 - ham
    true_pos  = 0
    true_neg  = 0
    false_neg = 0
    false_pos = 0
    
    for val, pred in zip(ground_truth,prediction):
        if (val == 0 and pred == 0):
            true_pos += 1
        elif (val == 1 and pred == 1):
            true_neg += 1
        elif(val == 0 and pred == 1):
            false_neg += 1
        elif (val == 1 and pred == 0):
            false_pos += 1
        else:
            print("An invalid value occured.")
            continue
    
    return true_pos, true_neg, false_neg, false_pos
        
def precision(true_pos, false_pos):
    return (true_pos/(true_pos + false_pos))
    
def recall(true_pos, false_neg):
    return (true_pos/(true_pos + false_neg))

In [18]:

def accuracyScore(y_true, y_pred, datatype): 
    # Calculate accuracy of the model's prediction

    true_pos, true_neg, false_neg, false_pos = calculate_confusion_matrix(y_true, y_pred, datatype)
    accuracy = (true_pos + true_neg)/(true_pos + true_neg + false_neg + false_pos )
    return accuracy

def f1Score(y_true, y_pred, datatype): 
    # Calculate F1 score of the model's prediction
    
    true_pos, true_neg, false_neg, false_pos = calculate_confusion_matrix(y_true, y_pred, datatype)
    precis = precision(true_pos, false_pos)
    recal = recall(true_pos, false_neg)
    f1 = 2 * ((precis * recal)/(precis + recal))
    return f1



In [19]:
print()
print("\nTraining accuracy: {:.3f}, F1 score: {:.3f}\n".format(accuracyScore(y_train, y_train_pred, "Training"), f1Score(y_train, y_train_pred, "Training")))
print("\nValidation accuracy: {:.3f}, "
      "F1 score: {:.3f}\n".format(accuracyScore(y_val, y_val_pred, "Validation"), f1Score(y_val, y_val_pred, "Validation")))
print("\nTest accuracy: {:.3f}, "
      "F1 score: {:.3f}\n".format(accuracyScore(y_test, y_test_pred, "Test"),f1Score(y_test, y_test_pred,"Test")))



Training accuracy: 0.964, F1 score: 0.979


Validation accuracy: 0.972, F1 score: 0.984


Test accuracy: 0.971, F1 score: 0.984



### Verify Accuracy and F-1 Score using Sklearn

            Sklearn F1 score is lower than what I got in my function. 
            It is due to the "Average" parameter that is being used. 
            If this parameter is changed to micro, F-1 Score increase. 
            If it is changed to "macro" it decreased. 
            Thus, "weighted" appears a good indicator. 

In [20]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print()
print("Accuracy Train: {:.3}".format(accuracy_score(y_train, y_train_pred)))
print("F1 score Train: {:.3}".format(f1_score(y_train, y_train_pred, average='weighted')))
print()

print("Accuracy Test: {:.3}".format(accuracy_score(y_test, y_test_pred)))
print("F1 score Test: {:.3}".format(f1_score(y_test, y_test_pred, average='weighted')))
print()

print("Accuracy Val: {:.3}".format(accuracy_score(y_val, y_val_pred)))
print("F1 score Val: {:.3}".format(f1_score(y_val, y_val_pred, average='weighted')))
print()


Accuracy Train: 0.964
F1 score Train: 0.962

Accuracy Test: 0.971
F1 score Test: 0.97

Accuracy Val: 0.972
F1 score Val: 0.971



## Qualitative Questions

### **Question.**
Is accuracy the metric that logistic regression optimizes while training? If no, which metric is optimized in logistic regression?

## **Your answer:** 

Accuracy is not the direct metric that logistic regression optimizes when training. Logistic Regression minimizes i.e. optimizes $\textbf{Negative Log Likelihood}$.

$$min \left(-\sum_{i=1}^m (y^{i} \times log(h_{\theta} (x^{i})) + (1-y^{(i)}) \times log(1 - h_{\theta}(x^{(i)})))\right) $$

In training, attempts are made to find the best parameters for our logistic regression Model that minimizes the cost function. Hence, the minimized cost function induces higher accuracy metric. So in short, no, accuracy is not what model is optimizing while training. It is the cost function of negative log likelihood that is being optimized which in turn enhances or increases the accuracy. 


### **Question.**
In general, does having 0.99 accuracy on test means that the model is great? If no, can you give an example of a case when the accuracy is high but the model is not good? (Hint: why do we use F1 score?)

### **Your answer:** 

Having too high of accuracy such as 0.99 can be very misleading. Accuracy is the measure of True positive and True negative cases i.e. all the correct or true predictions made by the model. While this may be great for cases like Spam detector, accuracy may not be the best metric for cases where False positive and False Negative hold a lot of value. For example, for cancer detection, model accuracy may be 0.99 on 1000 test data point, but it does also mean that 10 people were misclassified by the model which can be very deterimental. Or taking modern example, imagine 10,000 people are in the test dataset that are being predicted for "positive COVID." Our model's accuracy is 0.99. That means 100 people were misclassified as False negative or False positive. Due to 0.1 inaccuracy, COVID can spread to wider community. Thus, in this case, accuracy may not be the best metric or indicative of how good the model is. 

Due to this shortcoming of accuracy metric, F1 score hold significance as it is harmonic mean of precision and recall. F1 score penalizes or evaluates the model on False positives and False negatives. From the example above, F1 score will tell about the False negatives that were predicted by our model using the cancer patients data.  

Thus, in short, accuracy is a good metric when True Positives and True Negatives hold significane. However, when False negatives and False positives are significant, F1-score will be a better metric than accuracy.


### Exploration of predictions

### **Question**

Show a few examples with correctly predicted labels on the train and val sets.

In [21]:
'''Training Data'''
lst_Train_correct = []
lst_Train_incorrect = []

for tense, ori_label, pred_label in zip(train_texts, train_labels, y_train_pred):
    if int(ori_label) == int(pred_label):
        lst_Train_correct.append(tense)
    else:
        lst_Train_incorrect.append(tense)

print("\nTraining Data: Total Messages that were correctly classified: {} Misclassified: {}".\
      format(len(lst_Train_correct),len(lst_Train_incorrect)))


'''Change the index to see different classified sentences'''

print("\n\033[1mCorrectly Classified Examples from Training Data Split:\033[0m\n")
for i in lst_Train_correct[:11]:
    print(i)
print()


Training Data: Total Messages that were correctly classified: 3758 Misclassified: 142

Correctly Classified Examples from Training Data Split:

Convey my regards to him
[Û_] anyway, many good evenings to u! s
My sort code is  and acc no is . The bank is natwest. Can you reply to confirm i've sent this to the right person!
Sorry i din lock my keypad.
Ok i thk i got it. Then u wan me 2 come now or wat?
Oi when you gonna ring
Will be office around 4 pm. Now i am going hospital.
Have you heard about that job? I'm going to that wildlife talk again tonight if u want2come. Its that2worzels and a wizzle or whatever it is?! 
Oh my God. I'm almost home
No dear i do have free messages without any recharge. Hi hi hi
Actually fuck that, just do whatever, do find an excuse to be in tampa at some point before january though



In [22]:
lst_Val_correct = []
lst_Val_incorrect = []

for tense, ori_label, pred_label in zip(val_texts, val_labels, y_val_pred):
    if int(ori_label) == int(pred_label):
        lst_Val_correct.append(tense)
    else:
        lst_Val_incorrect.append(tense)

print("\nValidation Data: Total Messages that were correctly classified: {} Misclassified: {}".\
      format(len(lst_Val_correct),len(lst_Val_incorrect)))


'''Change the index to see different classified sentences'''

print("\n\033[1mCorrectly Classified Examples from Validation Data Split:\033[0m\n")
for i in lst_Val_correct[:10]:
    print(i)


Validation Data: Total Messages that were correctly classified: 813 Misclassified: 23

Correctly Classified Examples from Validation Data Split:

I just saw ron burgundy captaining a party boat so yeah
I love you. You set my soul on fire. It is not just a spark. But it is a flame. A big rawring flame. XoXo
Ok i vl..do u know i got adsense approved..
HCL chennai requires FRESHERS for voice process.Excellent english needed.Salary upto  &lt;#&gt; .Call Ms.Suman  &lt;#&gt;  for Telephonic interview -via Indyarocks.com
A boy was late 2 home. His father: \POWER OF FRNDSHIP\""
For ur chance to win å£250 cash every wk TXT: PLAY to 83370. T's&C's www.music-trivia.net custcare 08715705022, 1x150p/wk.
Badrith is only for chennai:)i will surely pick for us:)no competition for him.
Cool, we shall go and see, have to go to tip anyway. Are you at home, got something to drop in later? So lets go to town tonight! Maybe mum can take us in.
Good afternoon starshine! How's my boytoy? Does he crave me yet

### **Question** 

Print 10 examples from val set which were labeled incorrectly by the model. Why do you think the model got them wrong?

### **Your answer:** 

There can be different reasons for the misclassification. E.g. take the following example: 
                   
                FreeMsg Hey U, i just got 1 of these video/pic fones, reply WILD to this txt & ill 
                send U my pics, hurry up Im so bored at work xxx (18 150p/rcvd STOP2stop)

It was labeled as Spam, however, our model predicted it as Ham. It could be because most of the words in this message are regular tokens that are not being penalized. This message does not contain any phone number or any link to a site. Thus, it passes all of the additional features that were set in the Data Matrix. 

Or take this example: 

           Do you realize that in about 40 years, we'll have thousands of old ladies running around with tattoos?

In the dataset, this has been labeled as Spam, however, our model predicted it as HAM. Along with the reasons mentioned for the previous examples, more or less, all the token words in this message are regular vocab phrases. Thus, the weightage of the message in comparison to the corpus is tilted towards HAM rather than SPAM.

In [24]:
    
print("\n\033[1m10 MisClassified Examples from Validation Data Split:\033[0m\n")

for i in lst_Val_incorrect[:11]:
  print(i)
    


10 MisClassified Examples from Validation Data Split:

WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Hi, Mobile no.  &lt;#&gt;  has added you in their contact list on www.fullonsms.com It s a great place to send free sms to people For more visit fullonsms.com
Latest News! Police station toilet stolen, cops have nothing to go on!
This message is free. Welcome to the new & improved Sex & Dogging club! To unsubscribe from this service reply STOP. msgs@150p 18 only
This message is free. Welcome to the new & improved Sex & Dogging club! To unsubscribe from this service reply STOP. msgs@150p 18+only
Dorothy@kiefer.com (Bank of Granite issues Strong-Buy) EXPLOSIVE PICK FOR OUR MEMBERS *****UP OVER 300% *********** Nasdaq Symbol CDGT That is a $5.00 per..
Do you realize that in about 40 years, we'll have thousands of old ladies running around with tattoos?
Had your mobile 10 mths? 

## End of Code

# End of Notebook
